In [1]:
!pip install optuna-integration[xgboost] -q

In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from xgboost import XGBRegressor
import cupy as cp
import optuna

In [3]:
# Loading data 
X_train = pd.read_csv("../input/30-days-of-ml/train.csv").iloc[:100_000, :]
X_test = pd.read_csv("../input/30-days-of-ml/test.csv")

# Preparing data as a tabular matrix
y_train = X_train.target
X_train = X_train.set_index('id').drop('target', axis='columns')
X_test = X_test.set_index('id')

# Pointing out categorical features
categoricals = [item for item in X_train.columns if 'cat' in item]

# Dealing with categorical data using OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
X_train[categoricals] = ordinal_encoder.fit_transform(X_train[categoricals])
X_test[categoricals] = ordinal_encoder.transform(X_test[categoricals])

In [4]:
from optuna_integration import XGBoostPruningCallback

def objective(trial):
    
    params = {
            'learning_rate': trial.suggest_float("learning_rate", 0.01, 1.0, log=True),
            'reg_lambda': trial.suggest_float("reg_lambda", 1e-9, 100.0, log=True),
            'reg_alpha': trial.suggest_float("reg_alpha", 1e-9, 100.0, log=True),
            'subsample': trial.suggest_float("subsample", 0.1, 1.0),
            'colsample_bytree': trial.suggest_float("colsample_bytree", 0.1, 1.0),
            'max_depth': trial.suggest_int("max_depth", 1, 7),
            'min_child_weight': trial.suggest_int("min_child_weight", 1, 7),
            'gamma': trial.suggest_float("gamma", 0.1, 1.0, step=0.1)
    }

    model = XGBRegressor(
        random_state=0,
        tree_method = "hist", 
        device = "gpu",
        n_estimators=10_000,
        early_stopping_rounds=300,
        callbacks=[XGBoostPruningCallback(trial, 'validation_0-rmse')],
        **params,
    )
    
    model.fit(x, y, eval_set=[(x_val, y_val)], verbose=1000)
    preds = model.predict(cp.asarray(x_test.values))
    rmse = mean_squared_error(y_test, preds, squared=False)
    return rmse


In [5]:
x, x_val, y, y_val = train_test_split(X_train, y_train, random_state=0, test_size=0.2)
x, x_test, y, y_test = train_test_split(x, y, random_state=0, test_size=0.25)

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)

[I 2024-10-06 07:43:16,000] A new study created in memory with name: no-name-97bf573b-29a9-4036-b9ca-41a9de8d7ab7


[0]	validation_0-rmse:0.74298
[1000]	validation_0-rmse:0.72725
[2000]	validation_0-rmse:0.72434
[3000]	validation_0-rmse:0.72258
[4000]	validation_0-rmse:0.72187
[5000]	validation_0-rmse:0.72121
[6000]	validation_0-rmse:0.72081
[7000]	validation_0-rmse:0.72070
[7083]	validation_0-rmse:0.72070


[I 2024-10-06 07:43:30,261] Trial 0 finished with value: 0.7202722001387207 and parameters: {'learning_rate': 0.029142577621650317, 'reg_lambda': 0.006548888217760044, 'reg_alpha': 8.725393704306269, 'subsample': 0.2637717396252206, 'colsample_bytree': 0.10480925898128796, 'max_depth': 2, 'min_child_weight': 4, 'gamma': 0.1}. Best is trial 0 with value: 0.7202722001387207.


[0]	validation_0-rmse:0.74278
[1000]	validation_0-rmse:0.72893
[2000]	validation_0-rmse:0.72707
[3000]	validation_0-rmse:0.72583
[4000]	validation_0-rmse:0.72489
[5000]	validation_0-rmse:0.72416
[6000]	validation_0-rmse:0.72354
[7000]	validation_0-rmse:0.72305
[8000]	validation_0-rmse:0.72268
[9000]	validation_0-rmse:0.72237
[9999]	validation_0-rmse:0.72207


[I 2024-10-06 07:43:47,422] Trial 1 finished with value: 0.7221858632495973 and parameters: {'learning_rate': 0.05064403148512496, 'reg_lambda': 0.00037714890135213974, 'reg_alpha': 41.69327086483488, 'subsample': 0.675798981460365, 'colsample_bytree': 0.9214199806641051, 'max_depth': 1, 'min_child_weight': 3, 'gamma': 0.30000000000000004}. Best is trial 0 with value: 0.7202722001387207.


...

[I 2024-10-06 07:44:20,217] Trial 90 pruned. Trial was pruned at iteration 0.
[I 2024-10-06 07:44:20,421] Trial 91 pruned. Trial was pruned at iteration 0.
[I 2024-10-06 07:44:20,621] Trial 92 pruned. Trial was pruned at iteration 0.


[0]	validation_0-rmse:0.74038


[I 2024-10-06 07:44:20,831] Trial 93 pruned. Trial was pruned at iteration 0.
[I 2024-10-06 07:44:21,026] Trial 94 pruned. Trial was pruned at iteration 0.
[I 2024-10-06 07:44:21,222] Trial 95 pruned. Trial was pruned at iteration 0.
[I 2024-10-06 07:44:21,417] Trial 96 pruned. Trial was pruned at iteration 0.
[I 2024-10-06 07:44:21,614] Trial 97 pruned. Trial was pruned at iteration 0.
[I 2024-10-06 07:44:21,818] Trial 98 pruned. Trial was pruned at iteration 0.


[0]	validation_0-rmse:0.73824


[I 2024-10-06 07:44:22,031] Trial 99 pruned. Trial was pruned at iteration 7.


In [6]:
print(study.best_value)
print(study.best_params)

0.7202722001387207
{'learning_rate': 0.029142577621650317, 'reg_lambda': 0.006548888217760044, 'reg_alpha': 8.725393704306269, 'subsample': 0.2637717396252206, 'colsample_bytree': 0.10480925898128796, 'max_depth': 2, 'min_child_weight': 4, 'gamma': 0.1}
